In [1]:
# !pip install chromadb
# !pip install openai

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [4]:
import chromadb
import json
chroma_client = chromadb.Client()

In [5]:
f = open("dbinfo.json", "r")
dbinfo = json.loads(f.read())

f = open("dbrelationships.json", "r")
dbrelationships = json.loads(f.read())


In [6]:
ids = []
documents = []
for table in dbinfo["tables"]:
    ids.append(table['name'])
    documents.append(str(table))

In [7]:
import chromadb.utils.embedding_functions as embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=OPENAI_API_KEY,
                model_name="text-embedding-3-small"
            )
collection = chroma_client.create_collection(name="my_collection", embedding_function=openai_ef)

In [8]:
collection.add(
    documents=documents,
    ids=ids
)

In [30]:
results = collection.query(
    query_texts=["largest tables"], # Chroma will embed this for you
    n_results=2 # how many results to return
)


In [31]:
print(results['ids'][0])
print(results['documents'][0])

['table_entity', 'database_entity']
['{\'name\': \'table_entity\', \'properties\': {\'description\': \'Stores metadata about different tables within the databases, including their unique identifiers, names, and associated JSON metadata. For all the queries related to tables this should be used.\'}, \'refSql\': \'select * from table_entity\', \'columns\': [{\'name\': \'id\', \'type\': \'VARCHAR(36)\', \'notNull\': True, \'expression\': \'table_id\', \'properties\': {\'description\': \'Unique identifier for the table entity.\'}}, {\'name\': \'name\', \'type\': \'VARCHAR(256)\', \'notNull\': True, \'expression\': \'table_name\', \'properties\': {\'description\': \'The name of the table, used to identify it within the system.\'}}, {\'name\': \'json\', \'type\': \'JSON\', \'notNull\': True, \'expression\': \'table_metadata\', \'properties\': {\'description\': \'JSON data containing all metadata related to the table entity.\'}}, {\'name\': \'updatedAt\', \'type\': \'BIGINT UNSIGNED\', \'notN

In [32]:
tables = results['ids'][0]


In [45]:
",".join(tables)

'table_entity,database_entity'

In [36]:

def getRelationshipsForTables(tables):
    allrelationships = dbrelationships['relationships']
    relationships = []
    for rel in allrelationships:
        if (rel['fromTable'] in tables or rel['toTable'] in tables):
            relationships.append(rel)
    
    return relationships


In [37]:
relationships = getRelationshipsForTables(tables)

In [40]:
print(relationships)

[{'fromTable': 'tag_usage', 'columns': ['targetFQNHash'], 'toTable': 'table_entity', 'relatedColumns': ['fqnHash'], 'relationshipType': 'many-to-one', 'description': 'Tags are used to classify and describe tables in the metadata system.'}, {'fromTable': 'table_entity', 'columns': ['fqnHash'], 'toTable': 'tag_usage', 'relatedColumns': ['targetFQNHash'], 'relationshipType': 'one-to-many', 'description': 'Tags are used to categorize and describe tables.'}, {'fromTable': 'test_case', 'columns': ['entityFQN'], 'toTable': 'table_entity', 'relatedColumns': ['fqnHash'], 'relationshipType': 'many-to-one', 'description': 'The entityFQN in test_case links to various metadata entities such as table_entity to specify what the test case is validating.'}, {'fromTable': 'test_definition', 'columns': ['entityType'], 'toTable': 'table_entity', 'relatedColumns': ['fqnHash'], 'relationshipType': 'many-to-one', 'description': 'The entityType in test_definition references different entity types indicating w